<a href="https://colab.research.google.com/github/arthur-kt/PPP-Intertemporal-Converter/blob/main/PPP_converter_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPP Interporal Converter

## Parâmetros Iniciais

In [2]:
!pip install fredapi
!pip install pycountry

from fredapi import Fred
import pycountry
import pandas as pd
from datetime import datetime, timedelta
import requests
from collections import defaultdict

# Inicialize a chave do FRED
fred = Fred(api_key='11cb1e53b57458acaddd29a16e0efe1b')


# Obtenha o CPI dos EUA para ajustar pela inflação
cpi_data = fred.get_series('CPIAUCSL')
cpi_df = cpi_data.to_frame(name='CPI')
cpi_df.index = pd.to_datetime(cpi_df.index)
cpi_df_ano = cpi_df.resample('YE').mean()  # Média anual do CPI

# CPI atual para trazer o valor a 2024
cpi_atual = cpi_df_ano.loc['2024'].values[0]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.4 MB/s eta 0:00:00


## Importar dados PPP

In [3]:
# Importar dados PPP
url = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/eabb3ca9caaca37dc9d9f1ae46ad03f7bc699954/ppp_att.csv'
ppp_data =  pd.read_csv(url, delimiter=',')

## Obter taxa de conversão do dólar 2024

In [4]:
# Função para obter o código de moeda usando o código de país ISO2
def get_currency_code(country_code):
    try:
        country = pycountry.countries.get(alpha_2=country_code)
        currency = pycountry.currencies.get(numeric=country.numeric)
        return currency.alpha_3 if currency else None
    except AttributeError:
        print(f"Não foi possível encontrar a moeda para o código de país: {country_code}")
        return None

# Mudar ISO3 para ISO2
def iso3_to_iso2(iso3):
    try:
    # Convert iso3 to string and ensure it's uppercase
        iso3 = str(iso3).upper()
        return pycountry.countries.get(alpha_3=iso3).alpha_2
    except (LookupError, AttributeError):
        # Handle both LookupError and AttributeError
        return None

# Importar taxa de câmbio dos países
url2 = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/5b0b2f6071f19198b91a8ef10e2cfef3f3a1b91c/exchange_rates.csv'
exchange_rates24 =  pd.read_csv(url2)
exchange_rates24 = exchange_rates24[['Country Code', '2024 [2024]']]
exchange_rates24 = exchange_rates24.rename(columns={'2024 [2024]': '2024'})
exchange_rates24['Country Code'] = exchange_rates24['Country Code'].apply(iso3_to_iso2)

## Função Conversão

In [5]:
# Função para obter o PPP ajustado de um país em um ano específico a partir do CSV
def get_ppp_adjusted(country_code, year):
    ppp_row = ppp_data[(ppp_data['Country ID'] == country_code) & (ppp_data['Year'] == year)]
    return ppp_row.iloc[0]['PPP']

# Função para converter gasto local em dólar ajustado pela inflação dos EUA
def converter_para_dolar_corrente(row):
    # Para o ano 2023, usa os dados de PPP externos
    if row['Ano2'] == 0:
      if row['Ano'] == 2024:
        # Procura o exchange rate diretamente no dataframe exchange_rates24
            exchange_rate_row = exchange_rates24[exchange_rates24['Country Code'] == row['País_Moeda']]
            if not exchange_rate_row.empty:
                exchange_rate = exchange_rate_row.iloc[0]['2024']  # Obtém a taxa de câmbio para 2024
                exchange_rate = float(exchange_rate)
                gasto_em_dolar_2024 = row['Gasto_Local'] / exchange_rate
                return gasto_em_dolar_2024
      else:
          # Para demais anos
           ppp_adjusted = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
           gasto_em_dolar_ppp = row['Gasto_Local'] / ppp_adjusted

      # Ajuste pela inflação dos EUA
      cpi_ano_referencia = cpi_df_ano.loc[str(row['Ano'])].values[0]
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_ano_referencia)

    else:
      # Caso exista Ano2, calcula o PPP médio entre Ano1 e Ano2

      ppp_ano1 = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
      ppp_ano2 = get_ppp_adjusted(row['País_Moeda'], row['Ano2'])

      # Média dos gastos enre entre Ano1 e Ano2
      gasto_em_dolar_ppp = row['Gasto_Local'] / ((ppp_ano1 + ppp_ano2) / 2)


      # Ajuste pela inflação dos EUA usando CPI de Ano1 e Ano2

      # Obter o índice CPI de ambos os anos
      cpi_ano1 = cpi_df_ano.loc[str(row['Ano'])].values[0]
      cpi_ano2 = cpi_df_ano.loc[str(row['Ano2'])].values[0]

      # Calcula a média do CPI para ajuste pela inflação
      cpi_media = (cpi_ano1 + cpi_ano2) / 2

      # Ajusta o gasto usando a média do CPI
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_media)

    return float(gasto_ajustado)


## Função Formatação

In [6]:
# Função para formatar valores em milhões, bilhões ou trilhões para Moeda Local
def formatar_valor_moeda_local(valor, codigo_pais):
    currency_code = get_currency_code(codigo_pais)  # Obtém o código da moeda do país
    if not currency_code:
        currency_code = "Moeda desconhecida"  # Se não houver código, exibe "Moeda desconhecida"

    if valor >= 1_000_000_000_000:  # Trilhão
        return f"{currency_code} {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"{currency_code} {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"{currency_code} {valor / 1_000_000:.2f} milhões"
    else:
        return f"{currency_code} {valor:.2f}"

# Função para formatar valores em milhões, bilhões ou trilhões para Dólar Ajustado
def formatar_valor_dolar(valor):
    if valor >= 1_000_000_000_000:  # Trilhão
        return f"USD {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"USD {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"USD {valor / 1_000_000:.2f} milhões"
    else:
        return f"USD {valor:.2f}"

## Insira seus dados:

In [10]:
# Dados

## Insira em 'País_Moeda' o o país emissor da moeda utilizada (no caso de uso de moedas diferentes do país de origem), em 'País do Programa' o país de origem do Programa, em 'Ano' o ano da origem dos recursos, em 'Gasto_Local' a quantia dispendida, em 'Programa' o nome do Programa e em 'Descrição' uma breve descrição da função ou destinação dos valores.
## A coluna 'Ano2' serve para os programas que tem custos entre 2 diferentes anos, sendo necessário calcular a média do PPP e da inflação americana. Então, insira no 'Ano2' o último ano do programa, se não, preencha com 0.

dados = {
    'País_Moeda': ['US', #1
                   'US', #2
                   'US', #3
                   'US', #4
                   'US', #5
                   'US', #6
                   'US', #7
                   'US', #8
                   'US', #9
                   'US', #10
                   'US', #11
                   'US', #12
                   'US', #13
                   'US', #14
                   'US', #15
                   'US', #16
                   'US', #17
                   'GB', #18
                   'GB', #19
                   'GB', #20
                   'GB', #21
                   'GB', #22
                   'GB', #23
                   'GB', #24
                   'GB', #25
                   'GB', #26
                   'GB', #27
                   'GB', #28
                   'KR', #29
                   'KR', #30
                   'KR', #31
                   'KR', #32
                   'KR', #33
                   'KR', #34
                   'KR', #35
                   'KR', #36
                   'KR', #37
                   'KR', #38
                   'KR', #39
                   'KR', #40
                   'KR', #41

             ],  # Usando códigos ISO2 dos países

        'País do Programa': ['US', #1
                             'US', #2
                             'US', #3
                             'US', #4
                             'US', #5
                             'US', #6
                             'US', #7
                             'US', #8
                             'US', #9
                             'US', #10
                             'US', #11
                             'US', #12
                             'US', #13
                             'US', #14
                             'US', #15
                             'US', #16
                             'US', #17
                             'GB', #18
                             'GB', #19
                             'GB', #20
                             'GB', #21
                             'GB', #22
                             'GB', #23
                             'GB', #24
                             'GB', #25
                             'GB', #26
                             'GB', #27
                             'GB', #28
                             'KR', #29
                             'KR', #30
                             'KR', #31
                             'KR', #32
                             'KR', #33
                             'KR', #34
                             'KR', #35
                             'KR', #36
                             'KR', #37
                             'KR', #38
                             'KR', #39
                             'KR', #40
                             'KR', #41

             ],  # Usando códigos ISO2 dos países

    'Ano': [2020, #1
            2020, #2
            2020, #3
            2020, #4
            2024, #5
            2021, #6
            2022, #7
            2023, #8
            2024, #9
            2024, #10
            2022, #11
            2022, #12
            2019, #13
            2020, #14
            2021, #15
            2021, #16
            2021, #17
            2018, #18
            2018, #19
            2018, #20
            2018, #21
            2018, #22
            2021, #23
            2021, #24
            2021, #25
            2023, #26
            2023, #27
            2024, #28
            2020, #29
            2024, #30
            2014, #31
            2022, #32
            2014, #33
            2023, #34
            2017, #35
            2019, #36
            2019, #37
            2020, #38
            2020, #39
            2023, #40
            2023, #41
            ],

   'Ano2': [0, #1
            0, #2
            0, #3
            0, #4
            0, #5
            0, #6
            0, #7
            0, #8
            0, #9
            0, #10
            0, #11
            0, #12
            0, #13
            0, #14
            0, #15
            0, #16
            0, #17
            0, #18
            0, #19
            0, #20
            0, #21
            0, #22
            0, #23
            0, #24
            0, #25
            0, #26
            0, #27
            0, #28
            0, #29
            0, #30
            0, #31
            0, #32
            2022, #33
            0, #34
            0, #35
            0, #36
            0, #37
            0, #38
            0, #39
            0, #40
            0, #41
            ],

    'Gasto_Local': [2900000000, #1
                    208000000, #2
                    8400000000, #3
                    1513000000000, #4
                    31100000000, #5
                    382200000, #6
                    953200000, #7
                    1193200000, #8
                    993200000, #9
                    1195200000, #10
                    156800000, #11
                    88000000000, #12
                    7500000000, #13
                    5000000, #14
                    65300000000, #15
                    18230000000, #16
                    951800000, #17
                    21000000, #18
                    181000000, #19
                    66000000, #20
                    40000000, #21
                    280000000, #22
                    140000000, #23
                    200000000, #24
                    370000000, #25
                    4500000000, #26
                    520000000, #27
                    520000000, #28
                    849200000000, #29
                    471500000000, #30
                    26800000000, #31
                    293400000000, #32
                    1144900000000, #33
                    31500000000, #34
                    2600000000000, #35
                    4000000000000, #36
                    2000000000000, #37
                    2800000000000, #38
                    595500000000, #39
                    500000000000, #40
                    261600000000, #41


                    ],  # Gasto em moeda local

    'Programa': ['Defense Production Act (DPA)', #1
                 'Defense Production Act (DPA)', #2
                 'Defense Production Act (DPA)', #3
                 'Defense Production Act (DPA)', #4
                 'Global Health Security', #5
                 'Global Health Security', #6
                 'Global Health Security', #7
                 'Global Health Security', #8
                 'Global Health Security', #9
                 'Global Health Security', #10
                 'NATIONAL BIODEFENSE STRATEGY AND IMPLEMENTATION', #11
                 'NATIONAL BIODEFENSE STRATEGY AND IMPLEMENTATION', #12
                 'National Biodefense Strategy', #13
                 'National Biodefense Strategy', #14
                 'American Pandemic Preparedness: Transforming Our Capabilities', #15
                 'Operação Warp Speed (OWS)', #16
                 'Operação Warp Speed (OWS)', #17
                 'Industrial Strategy Challenge Fund: Leading-edge healthcare', #18
                 'Industrial Strategy Challenge Fund: Leading-edge healthcare', #19
                 'Industrial Strategy Challenge Fund: Leading-edge healthcare', #20
                 'Industrial Strategy Challenge Fund: Healthy ageing', #21
                 'Industrial Strategy Challenge Fund: Healthy ageing', #22
                 'Life Sciences Vision', #23
                 'Life Sciences Vision', #24
                 'Life Sciences Vision', #25
                 'Advanced Manufacturing Plan', #26
                 'Advanced Manufacturing Plan', #27
                 'Life Sciences Innovative Manufacturing Fund', #28
                 'MSS - Ministry of SMEs and Startups', #29
                 'Tech Incubation Program for Start-ups (TIPS)', #30
                 'Tech Incubation Program for Start-ups (TIPS)', #31
                 'Tech Incubation Program for Start-ups (TIPS)', #32
                 'Tech Incubation Program for Start-ups (TIPS)', #33
                 'Deeptech Tech Incubator Program for Startup', #34
                 'Bio-Health Industry Innovation Strategy', #35
                 'Bio-Health Industry Innovation Strategy', #36
                 'Bio-Health Industry Innovation Strategy', #37
                 'Bio-Health Industry Innovation Strategy', #38
                 'Bio-Health Industry Innovation Strategy', #39
                 'K-Bio Vaccine Fund', #40
                 'K-Bio Vaccine Fund', #41


                 ],  # Nomes dos programas

    'Descrição': ['Title I: Used to purchase over 187,000 ventilators by the end of the year', #1
                  'Title III: Increase capacity for nasal swabs and respirators in limited amounts', #2
                  'Obrigou este valor à compra de fornecimentos e à reposição do Estoque Nacional Estratégico', #3
                  'Award for domestic production expansion projects for medical supplies using CARES Act appropriations', #4
                  'Experts estimate this additional amount annually, of which two-thirds would come from domestic financing and one-third from external financing, is needed to strengthen outbreak and pandemic prevention, preparedness, and response capacities.', #5
                  'USAID/State and CDC GHS Funding - FY 2021 Enacted', #6
                  'USAID/State and CDC GHS Funding - FY 2022 Enacted', #7
                  'USAID/State and CDC GHS Funding - FY 2023 Enacted', #8
                  'USAID/State and CDC GHS Funding - FY 2024 Enacted', #9
                  'USAID/State and CDC GHS Funding - FY 2025 President Budget', #10
                  'Beyond the Needle program e Coalition for Epidemic Preparedness Innovations', #11
                  'President’s request over five years for pandemic preparedness and biodefense', #12
                  'Reported spending on biodefense across all D / As for 2019', #13
                  'Orçamento pedido pelo HSS para implementar a National Biodefense Strategy. No entanto, o orçamento de 2020 não incluíu esse valor requisitado.', #14
                  'Custo estimado para período entre 7 e 10 anos', #15
                  'Valor dos contratos de Vaccine Candidates Supported by BARDA and Other Federal Agencies', #16
                  'Valor dos Federal Government Contracts for Ancillary COVID-19 Vaccine Supplies', #17
                  'Financiamento para 3 centros de tratamento de terapias avançadas', #18
                  'Desafio para desenvolvimento de novas medicinas, vacinas e outros produtos da saúde, a ser investido em 4 anos', #19
                  'Investimento no Vaccines Manufacturing Innovation Centre', #20
                  'Investimento no UK Dementia Research Institute (UKDRI)', #21
                  'Investimento em centros regionais de exelência e um programa de uso de dados para medicina de precisão e diagnósticos preventivos', #22
                  'NHS AI Health and Care Award delivered by the NHSX AI Lab in partnership with the AAC', #23
                  'Lançamento do Life Sciences Investment Programme (LSIP)', #24
                  'Aumento da capacidade de manufatura de vacinas e fundo de apoio a manufatura', #25
                  'Montante total disponibilizado para financiamento no âmbito do plano a partir de 2025, por cinco anos (2025-2030)', #26
                  'Parte do plano destinada às indústrias da ciências da vida', #27
                  'Segunda Fase (Valor atualizado out/24)', #28
                  'Orçamento para o ministério em 2020', #29
                  'Suporte planejado para 1925 empresas', #30
                  'Orçamento TIPS em 2014', #31
                  'Orçamento TIPS em 2022', #32
                  'Alocação governamental acumulada entre 2014 e 2022', #33
                  'Anunciado que o programa suportaria 120 projetos com este orçamento', #34
                  'Antes da estratégia: investimentos em P&D na biotecnologia e no setor da saúde em 2017', #35
                  'Meta de investimentos em P&D na biotecnologia e no setor da saúde para 2025', #36
                  'Policy loans a year for the next five years', #37
                  'O governo planeia investir no desenvolvimento de dois projectos interagências – desenvolvimento de novos medicamentos e desenvolvimento de tecnologia de medicina regenerativa. Os dois projetos nacionais fazem parte da Bio-Health Industry Innovation Strategy', #38
                  'Investimento na área de medicina regenerativa por 10 anos a partir de 2021', #39
                  'A futura tecnologia médica será financiada através da criação do K-Bio Vaccine Fund, no primeiro semestre de 2023, para expandir o investimento no desenvolvimento de vacinas e terapêuticos.', #40
                  'K-Bio Vaccine Fund Nos. 1 e 2 arrecadaram este valor em investimentos.', #41
                  ]  # Descrições dos programas
}

#Check se todos dados estão com o mesmo número de entradas
print(len(dados['Programa']),
len(dados['Descrição']),
len(dados['País_Moeda']),
len(dados['País do Programa']),
len(dados['Ano']),
len(dados['Ano2']),
len(dados['Gasto_Local']))

df = pd.DataFrame(dados)

33 33 33 33 33 33 33


## Conversor

In [9]:
# Aplicar a função para cada linha no DataFrame
df['Gasto_Convertido_Dolar_2024'] = df.apply(converter_para_dolar_corrente, axis=1)

# Aplicar as novas funções para as colunas 'Gasto_Local' e 'Gasto_Convertido_Dolar_2024'
df['Gasto Local'] = df.apply(lambda row: formatar_valor_moeda_local(row['Gasto_Local'], row['País_Moeda']), axis=1)
df['Gasto Convertido US$ 2024'] = df['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

# Exibir o DataFrame com as colunas formatadas
df[['País do Programa', 'Programa', 'Descrição', 'Ano', 'Ano2', 'Gasto Local', 'Gasto Convertido US$ 2024']]

<ipython-input-5-c61517678bf2>:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(gasto_ajustado)


,País do Programa,Programa,Descrição,Ano,Ano2,Gasto Local,Gasto Convertido US$ 2024
0,US,Defense Production Act (DPA),"Title I: Used to purchase over 187,000 ventila...",2020,0,USD 2.90 bilhões,USD 3.50 bilhões
1,US,Defense Production Act (DPA),Title III: Increase capacity for nasal swabs a...,2020,0,USD 208.00 milhões,USD 251.32 milhões
2,US,Defense Production Act (DPA),Obrigou este valor à compra de fornecimentos e...,2020,0,USD 8.40 bilhões,USD 10.15 bilhões
3,US,Defense Production Act (DPA),Award for domestic production expansion projec...,2020,0,USD 1.51 trilhões,USD 1.83 trilhões
4,US,Global Health Security,Experts estimate this additional amount annual...,2024,0,USD 31.10 bilhões,USD 31.10 bilhões
5,US,Global Health Security,USAID/State and CDC GHS Funding - FY 2021 Enacted,2021,0,USD 382.20 milhões,USD 441.14 milhões
6,US,Global Health Security,USAID/State and CDC GHS Funding - FY 2022 Enacted,2022,0,USD 953.20 milhões,USD 1.02 bilhões
7,US,Global Health Security,USAID/State and CDC GHS Funding - FY 2023 Enacted,2023,0,USD 1.19 bilhões,USD 1.22 bilhões
8,US,Global Health Security,USAID/State and CDC GHS Funding - FY 2024 Enacted,2024,0,USD 993.20 milhões,USD 993.20 milhões
9,US,Global Health Security,USAID/State and CDC GHS Funding - FY 2025 Pres...,2024,0,USD 1.20 bilhões,USD 1.20 bilhões
